# Problem 1 
We denoise Goldhill and Lena images with 3 different methods:

- Gaussian
- Median
- Bilateral

Then also apply the following sharpening filters to each image:
- Laplacian



In [1]:
# imports
import cv2
import numpy as np
import os

In [2]:
# image imports
lena_original_path = os.path.join('imgs', 'lena_ori.png')
lena_noisy_path = os.path.join('imgs', 'lena_noisy.png')
goldhill_original_path = os.path.join('imgs', 'Goldhill_2.jpg')
goldhill_noisy_path = os.path.join('imgs', 'Goldhill_2noisy.jpg')

# read images
lena_original = cv2.imread(lena_original_path)
lena_noisy = cv2.imread(lena_noisy_path)
goldhill_original = cv2.imread(goldhill_original_path)
goldhill_noisy = cv2.imread(goldhill_noisy_path)

sharpening_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])

# Process images: Goldhill 2 first, then Lena (to match table order)
image_configs = [
    ("Goldhill", goldhill_original, goldhill_noisy),
    ("Lena", lena_original, lena_noisy),
]

results = []

for name, original, noisy in image_configs:
    if noisy.shape != original.shape:
        noisy = cv2.resize(noisy, (original.shape[1], original.shape[0]))

    # PSNR: Noisy image vs original
    psnr_noisy = cv2.PSNR(original, noisy)

    # 1. Gaussian denoising
    gaussian_denoised = cv2.GaussianBlur(noisy, (5, 5), 0)
    psnr_gaussian_denoised = cv2.PSNR(original, gaussian_denoised)
    gaussian_sharpened = cv2.filter2D(gaussian_denoised, -1, sharpening_kernel)
    psnr_gaussian_sharpened = cv2.PSNR(original, gaussian_sharpened)

    # 2. Median denoising
    median_denoised = cv2.medianBlur(noisy, 5)
    psnr_median_denoised = cv2.PSNR(original, median_denoised)
    median_sharpened = cv2.filter2D(median_denoised, -1, sharpening_kernel)
    psnr_median_sharpened = cv2.PSNR(original, median_sharpened)

    # 3. Bilateral denoising
    bilateral_denoised = cv2.bilateralFilter(noisy, 9, 75, 75)
    psnr_bilateral_denoised = cv2.PSNR(original, bilateral_denoised)
    bilateral_sharpened = cv2.filter2D(bilateral_denoised, -1, sharpening_kernel)
    psnr_bilateral_sharpened = cv2.PSNR(original, bilateral_sharpened)

    results.append({
        "name": name,
        "psnr_noisy": psnr_noisy,
        "methods": [
            ("Gaussian", psnr_gaussian_denoised, psnr_gaussian_sharpened),
            ("Median", psnr_median_denoised, psnr_median_sharpened),
            ("Bilateral", psnr_bilateral_denoised, psnr_bilateral_sharpened),
        ],
    })

# Print table for easy copy-paste
print("PSNR - Table Results")
print("=" * 90)
print(f"{'Noisy Image':<12} {'Method':<12} {'Noisy (vs orig)':<18} {'Denoised only':<18} {'Denoised+Sharpened':<20}")
print("-" * 90)
for r in results:
    for i, (method, psnr_denoised, psnr_sharpened) in enumerate(r["methods"]):
        print(f"{r['name']:<12} {method:<12} {r['psnr_noisy']:<18.2f} {psnr_denoised:<18.2f} {psnr_sharpened:<20.2f}")
    print("-" * 90)



PSNR - Table Results
Noisy Image  Method       Noisy (vs orig)    Denoised only      Denoised+Sharpened  
------------------------------------------------------------------------------------------
Goldhill     Gaussian     28.76              28.46              28.61               
Goldhill     Median       28.76              27.98              26.73               
Goldhill     Bilateral    28.76              28.28              27.95               
------------------------------------------------------------------------------------------
Lena         Gaussian     20.13              28.58              25.08               
Lena         Median       20.13              27.76              22.40               
Lena         Bilateral    20.13              24.37              11.79               
------------------------------------------------------------------------------------------


In [ ]:
# Save the 6 sharpened final images for upload
output_dir = 'imgs'
for name, original, noisy in image_configs:
    prefix = "goldhill" if "Goldhill" in name else "lena"
    gaussian_sharpened = cv2.filter2D(cv2.GaussianBlur(noisy, (5, 5), 0), -1, sharpening_kernel)
    median_sharpened = cv2.filter2D(cv2.medianBlur(noisy, 5), -1, sharpening_kernel)
    bilateral_sharpened = cv2.filter2D(cv2.bilateralFilter(noisy, 9, 75, 75), -1, sharpening_kernel)
    cv2.imwrite(os.path.join(output_dir, f'{prefix}_gaussian_sharpened.png'), gaussian_sharpened)
    cv2.imwrite(os.path.join(output_dir, f'{prefix}_median_sharpened.png'), median_sharpened)
    cv2.imwrite(os.path.join(output_dir, f'{prefix}_bilateral_sharpened.png'), bilateral_sharpened)
print("Saved 6 sharpened images to imgs/")